## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-05-18-56-58 +0000,nyt,Venezuela Live Updates: Maduro Tells U.S. Judg...,https://www.nytimes.com/live/2026/01/05/world/...
1,2026-01-05-18-56-38 +0000,nyt,Hegseth Starts Proceedings Against Sen. Mark K...,https://www.nytimes.com/2026/01/05/us/politics...
2,2026-01-05-18-53-43 +0000,wapo,Live updates: Venezuelan leader Maduro pleads ...,https://www.washingtonpost.com/world/2026/01/0...
3,2026-01-05-18-53-43 +0000,wapo,Pope Leo XIV is praying for peace in Venezuela...,https://www.washingtonpost.com
4,2026-01-05-18-52-00 +0000,wsj,The Pentagon is moving to demote Sen. Mark Kel...,https://www.wsj.com/politics/hegseth-moves-to-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2484/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,venezuela,66
123,trump,49
3,maduro,47
251,new,22
224,oil,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
248,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...,214
158,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,175
161,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,175
135,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...,163
257,2026-01-04-19-52-56 +0000,nyt,Why Trump Removed Venezuela’s Maduro,https://www.nytimes.com/video/world/americas/1...,163


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
248,214,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...
35,74,2026-01-05-18-04-32 +0000,nypost,Toppled Venezuelan dictator Nicolás Maduro dec...,https://nypost.com/2026/01/05/us-news/toppled-...
171,65,2026-01-05-07-55-46 +0000,nypost,Gov. Tim Walz expected to drop bid for re-elec...,https://nypost.com/2026/01/05/us-news/gov-tim-...
101,47,2026-01-05-15-07-08 +0000,nypost,"Chevron, energy stocks soar after US capture o...",https://nypost.com/2026/01/05/business/chevron...
135,38,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...
97,36,2026-01-05-15-15-57 +0000,nypost,"Sen. Mark Kelly at risk of demotion, cut in mi...",https://nypost.com/2026/01/05/us-news/sen-mark...
216,35,2026-01-04-23-19-44 +0000,nypost,Freed Israeli hostage details fear of being ma...,https://nypost.com/2026/01/04/world-news/freed...
256,31,2026-01-04-19-53-22 +0000,nypost,Owner of Swiss nightclub where deadly fire bro...,https://nypost.com/2026/01/04/world-news/owner...
92,30,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
181,27,2026-01-05-04-48-20 +0000,latimes,Emotional vigil held for man killed by off-dut...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
